In [3]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

# Load merged features with target_return_5d
df = pd.read_sql("SELECT * FROM merged_features", engine, parse_dates=["date"])

# Check if target_return_5d is present
if 'target_return_5d' not in df.columns:
    raise Exception("target_return_5d not found. Run label generation first.")

# Encode signals
def encode_signal(r):
    if r > 0.015:
        return 'BUY'
    elif r < -0.015:
        return 'SELL'
    else:
        return 'HOLD'

df['trade_signal'] = df['target_return_5d'].apply(encode_signal)

# Save back to DB as final_features
df.to_sql("merged_features", engine, if_exists="replace", index=False)
print(f"✅ Trade signals encoded and saved to final_features ({len(df)} rows)")

✅ Trade signals encoded and saved to final_features (6643 rows)
